GERAR O LIVRO COMO ARQUIVO PARA O GEPHI

In [17]:
import spacy
import networkx as nx
import requests
from collections import defaultdict

# Baixar o livro do Project Gutenberg
url = "https://www.gutenberg.org/cache/epub/24344/pg24344.txt"
response = requests.get(url)
text = response.text

# Carregar o modelo de spaCy
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Dicionários para armazenar entidades
entities = defaultdict(set)

# Processar o texto para identificar categorias
for ent in doc.ents:
    if ent.label_ in ["PERSON", "ORG", "GPE"]:
        entities[ent.label_].add(ent.text)

# Criar grafo com NetworkX
G = nx.Graph()

# Adicionar nós ao grafo
for label, names in entities.items():
    for name in names:
        G.add_node(name, label=label)

# Criar arestas baseadas em coocorrência no mesmo parágrafo
paragraphs = text.split("\n\n")
for paragraph in paragraphs:
    paragraph_doc = nlp(paragraph)
    paragraph_entities = [ent.text for ent in paragraph_doc.ents if ent.label_ in ["PERSON", "ORG", "GPE"]]
    for i, entity1 in enumerate(paragraph_entities):
        for entity2 in paragraph_entities[i + 1:]:
            if G.has_edge(entity1, entity2):
                G[entity1][entity2]["weight"] += 1
            else:
                G.add_edge(entity1, entity2, weight=1)

# Exportar para GEXF
nx.write_gexf(G, "graph_entities.gexf")

print("Arquivo 'graph_entities.gexf' gerado com sucesso!")


Arquivo 'graph_entities.gexf' gerado com sucesso!
